<a href="https://colab.research.google.com/github/huynguyenfts/Example-sentiment-comment/blob/master/Example_Sentiment_Comment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment comment by ML cơ bản**

**1. Clone git repo **

In [1]:
!pip install -q xlrd
!git clone https://github.com/huynguyenfts/Example-sentiment-comment.git

Cloning into 'Example-sentiment-comment'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 34 (delta 14), reused 11 (delta 2), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [2]:
%cd Example-sentiment-comment
!ls

/content/Example-sentiment-comment
basedline_model.py		 sample.csv		  test.crash
Example_Sentiment_Comment.ipynb  sentiment_comment.ipynb  train.crash
README.md			 submission.csv


**2. Import thư viện**

In [0]:
import pandas as pd
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

**3. Xử Lý data**

In [0]:
class DataSource(object):

    def _load_raw_data(self, filename, is_train=True):

        a = []
        b = []

        regex = 'train_'
        if not is_train:
            regex = 'test_'

        with open(filename, 'r') as file:
            for line in file:
                if regex in line:
                    b.append(a)
                    a = [line]
                elif line != '\n':
                    a.append(line)

        b.append(a)

        return b[1:]

    def _create_row(self, sample, is_train=True, dont_remove_space=True):

        d = {}
        d['id'] = sample[0].replace('\n', '')
        review = ""

        if is_train:
            for clause in sample[1:-1]:
                review += clause.strip()
#                 review += clause.replace('\n', '').strip()
#                 review += clause.replace('\n', '').replace('.', ' ').replace(',', ' ').strip()
            d['label'] = int(sample[-1].replace('\n', ''))
        else:
            for clause in sample[1:]:
                review += clause.replace('\n', '').strip()

        d['review'] = review

        return d

    def load_data(self, filename, is_train=True):

        raw_data = self._load_raw_data(filename, is_train)
        lst = []

        for row in raw_data:
            lst.append(self._create_row(row, is_train))

        return lst


In [0]:
ds = DataSource()
train_data = pd.DataFrame(ds.load_data('train.crash'))
test_data = pd.DataFrame(ds.load_data('test.crash', is_train=False))

In [6]:
print("Train data")
train_data.head(20)

Train data


,id,label,review
0,train_000000,0,"""Dung dc sp tot cam onshop Đóng gói sản phẩm r..."
1,train_000001,0,""" Chất lượng sản phẩm tuyệt vời Son mịn nhưn..."
2,train_000002,0,""" Chất lượng sản phẩm tuyệt vời nhưng k có hộp..."
3,train_000003,1,""":(( Mình hơi thất vọng 1 chút vì mình đã kỳ v..."
4,train_000004,1,"""Lần trước mình mua áo gió màu hồng rất ok mà ..."
5,train_000005,0,""" Chất lượng sản phẩm tuyệt vời có điều không ..."
6,train_000006,0,"""Đã nhận đc hàng rất nhanh mới đặt buổi tối mà..."
7,train_000007,1,"""Các siêu phẩm thấy cấu hình toàn tựa tựa nhau..."
8,train_000008,0,"""Hàng ship nhanh chất lượng tốt tư vấn nhiệt..."
9,train_000009,1,"""Đồng hồ đẹp nhưng 1 cái đứt dây 1 cái k chạy..."


In [0]:
print("Test data")
test_data.head(20)

Test data


,id,review
0,test_000000,"""Chưa dùng thử nên chưa biết"""
1,test_000001,""" Không đáng tiềnVì ngay đợt sale nên mới mua ..."
2,test_000002,"""Cám ơn shop. Đóng gói sản phẩm rất đẹp và chắ..."
3,test_000003,"""Vải đẹp.phom oki luôn.quá ưng"""
4,test_000004,"""Chuẩn hàng đóng gói đẹp"""
5,test_000005,""" Đóng gói sản phẩm rất đẹp và chắc chắn Shop ..."
6,test_000006,"""Sau khi đọc xong cuốn truyện thì cảm xú..."
7,test_000007,"""Chỉ cảm ứng khi gần dây điện ổ cắm ko có vật ..."
8,test_000008,"""Tệ😡 Sản phẩm đứt chỉ tùm lum😡 Rách quá trời c..."
9,test_000009,"""Shop Chất lượng sản phẩm rất kém Shop phục v..."


**4. Trích chọn đặc trưng**

In [0]:
vectorizer = TfidfVectorizer(max_features=100000, ngram_range=(1, 3))

x_train, x_val, y_train, y_val = train_test_split(train_data.review, train_data.label, test_size=0.2,
                                                  random_state=42)

vectorizer.fit(x_train)
x_tfidf_train = vectorizer.transform(x_train)
x_tfidf_val = vectorizer.transform(x_val)


**5. Build model**

In [18]:
model = LogisticRegression()
print("Train model.......")
sentiment_fit = model.fit(x_tfidf_train, y_train)
print("Predict ......")
y_pred = sentiment_fit.predict(x_tfidf_val)
accuracy = accuracy_score(y_val, y_pred)
print("accuracy score: {0:.2f}%".format(accuracy * 100))

Train model.......


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Predict ......
accuracy score: 88.63%


# **6. Dự đoán**

In [0]:
x_tfidf_test = vectorizer.transform(test_data.review)
y_predict = sentiment_fit.predict(x_tfidf_test)
test_data['label'] = y_predict
test_data[['id', 'label']].to_csv('sample.csv', index=False)

In [20]:
print(test_data)

                id                                             review  label
0      test_000000                      "Chưa dùng thử nên chưa biết"      0
1      test_000001  " Không đáng tiềnVì ngay đợt sale nên mới mua ...      1
2      test_000002  "Cám ơn shop. Đóng gói sản phẩm rất đẹp và chắ...      0
3      test_000003                    "Vải đẹp.phom oki luôn.quá ưng"      0
4      test_000004                          "Chuẩn hàng đóng gói đẹp"      0
5      test_000005  " Đóng gói sản phẩm rất đẹp và chắc chắn Shop ...      0
6      test_000006  "Sau khi đọc xong cuốn truyện thì cảm xú...      1
7      test_000007  "Chỉ cảm ứng khi gần dây điện ổ cắm ko có vật ...      1
8      test_000008  "Tệ😡 Sản phẩm đứt chỉ tùm lum😡 Rách quá trời c...      1
9      test_000009  "Shop  Chất lượng sản phẩm rất kém Shop phục v...      1
10     test_000010                     "Ad chỉ em cách chỉnh ngày vs"      1
11     test_000011                               "Cắm phát nhận luôn"      0

** 7.Test trực quan**

In [24]:
testTQ_data = {'review': ["hàng đẹp", "hàng tàu", "hàng khựa", "xấu quá", "nhìn kì quá", "không đúng size", "thái độ lồi lõm quá", "shiper đẹp trai", "khá bảnh", "lởm", "trá hình", "kèo thơm"]}
test_review = pd.DataFrame(data=testTQ_data)
x_test = vectorizer.transform(test_review.review)
y_test_predict = sentiment_fit.predict(x_test)
test_review['label'] = y_test_predict
print(test_review)

                 review  label
0              hàng đẹp      0
1              hàng tàu      1
2             hàng khựa      0
3               xấu quá      1
4           nhìn kì quá      1
5       không đúng size      1
6   thái độ lồi lõm quá      1
7       shiper đẹp trai      0
8              khá bảnh      0
9                   lởm      1
10             trá hình      0
11             kèo thơm      0
